# pytorch-yolo2

### create the subfolder

In [1]:
! mkdir onnx

ref: https://github.com/longcw/yolo2-pytorch

### download weight

In [3]:
! wget http://pjreddie.com/media/files/yolo.weights

--2018-07-09 04:07:36--  http://pjreddie.com/media/files/yolo.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.3.39
Connecting to pjreddie.com (pjreddie.com)|128.208.3.39|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://pjreddie.com/media/files/yolo.weights [following]
--2018-07-09 04:07:36--  https://pjreddie.com/media/files/yolo.weights
Connecting to pjreddie.com (pjreddie.com)|128.208.3.39|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 203934260 (194M) [application/octet-stream]
Saving to: ‘yolo.weights’

yolo.weights        100%[===================>] 194.49M  2.45MB/s    in 94s     

2018-07-09 04:09:11 (2.08 MB/s) - ‘yolo.weights’ saved [203934260/203934260]



In [1]:
!ls

1.yolo2_pytorch_onnx_save_model.ipynb	       detect.py	 partial.py
2.yolo2_pytorch_onnx_load_model.ipynb	       eval.py		 __pycache__
3.vggnet_onnx.ipynb			       FocalLoss.py	 README.md
4.Inference_test.ipynb			       GUI.py		 recall.py
5.Visualization.ipynb			       how_to_debug.md	 region_loss.py
6. IR_info_Extraction&Calculate.ipynb	       image.py		 scripts
7. IR_info_Extraction&Matching_Subgroup.ipynb  Inference.py	 svg
cfg					       IR_Extraction.py  tools
cfg.py					       issue.md		 train.py
darknet.py				       layers		 utils.py
data					       LICENSE		 valid.py
dataset.py				       models		 yolo.pth
debug.py				       net_drawer.py	 yolo.weights
demo_fig				       onnx
demo.py					       Onnx.py


### try predict

In [1]:
! python detect.py './cfg/yolo.cfg' './yolo.weights' './data/dog.jpg'

layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  32
    1 max          2 x 2 / 2   416 x 416 x  32   ->   208 x 208 x  32
    2 conv     64  3 x 3 / 1   208 x 208 x  32   ->   208 x 208 x  64
    3 max          2 x 2 / 2   208 x 208 x  64   ->   104 x 104 x  64
    4 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128
    5 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64
    6 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128
    7 max          2 x 2 / 2   104 x 104 x 128   ->    52 x  52 x 128
    8 conv    256  3 x 3 / 1    52 x  52 x 128   ->    52 x  52 x 256
    9 conv    128  1 x 1 / 1    52 x  52 x 256   ->    52 x  52 x 128
   10 conv    256  3 x 3 / 1    52 x  52 x 128   ->    52 x  52 x 256
   11 max          2 x 2 / 2    52 x  52 x 256   ->    26 x  26 x 256
   12 conv    512  3 x 3 / 1    26 x  26 x 256   ->    26 x  26 x 512
   13 conv    256  1 x 

### get model

In [3]:
from darknet import Darknet
from cfg import parse_cfg

In [4]:
cfgfile =  './cfg/yolo.cfg' 
weightfile =  './yolo.weights'
imgfile = './data/dog.jpg'

In [6]:
m = Darknet(cfgfile)

In [7]:
m.load_weights(weightfile)
print('Loading weights from %s... Done!' % (weightfile))

Loading weights from ./yolo.weights... Done!


/home/paperspace/yolo2_onnx/cfg.py:179: UserWarning: src is not broadcastable to dst, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  conv_model.weight.data.copy_(torch.from_numpy(buf[start:start+num_w])); start = start + num_w #20180615 modified for <= torch v0.3.0 post4
/home/paperspace/yolo2_onnx/cfg.py:158: UserWarning: src is not broadcastable to dst, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  conv_model.weight.data.copy_(torch.from_numpy(buf[start:start+num_w])); start = start + num_w #20180615 modified for <= torch v0.3.0 post4


### save detection information

In [5]:
import pickle
op_dict = {
    'num_classes':m.num_classes,
    'anchors':m.anchors,
    'num_anchors':m.num_anchors
}
pickle.dump(op_dict, open('detection_information.pkl','wb'))

# use Onnx to convert model

ref: https://github.com/onnx/tutorials/blob/master/tutorials/PytorchOnnxExport.ipynb

In [28]:
torch.save(m, './yolo.pth')

In [8]:
import torch.onnx
from torch.autograd import Variable

# Standard ImageNet input - 3 channels, 224x224,
# values don't matter as we care about network structure.
# But they can also be real inputs.
dummy_input = Variable(torch.randn(1, 3, 416, 416))
# Obtain your model, it can be also constructed in your script explicitly
model = m
# Invoke export
torch.onnx.export(model, dummy_input, "onnx/yolo2.onnx")